<a href="https://colab.research.google.com/github/TOBYKRISHNA/Time-crime/blob/main/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install dash
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import dash

# Load datasets
train_file_path = "Train.xlsx"
test_file_path = "Test.csv"
train_df = pd.read_excel(train_file_path)
test_df = pd.read_csv(test_file_path)

# Data Preprocessing
train_df.dropna(inplace=True)  # Remove missing values
test_df.dropna(inplace=True)

# Convert Date column to datetime format
df = pd.concat([train_df, test_df], ignore_index=True)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df.dropna(subset=['Date'], inplace=True)  # Drop invalid dates

# Aggregate data by Date and Crime Type
df['Incident_Counts'] = df.groupby(['Date', 'TYPE'])['TYPE'].transform('count')
df = df.drop_duplicates(subset=['Date', 'TYPE'])  # Keep unique date-type pairs

# Exploratory Data Analysis (EDA)
crime_summary = df.groupby('TYPE')['Incident_Counts'].sum().reset_index()
print("Crime Summary:")
print(crime_summary.head())

# Initialize Dash app
app = Dash(__name__)
server = app.server

app.layout = html.Div([
    html.H1("Crime Data Dashboard", style={'textAlign': 'center'}),

    dcc.Dropdown(
        id='crime_type',
        options=[{'label': c, 'value': c} for c in df['TYPE'].unique()],
        value=df['TYPE'].unique()[0],
        multi=False
    ),

    dcc.Graph(id='time_series'),
    dcc.Graph(id='crime_bar'),
    dcc.Graph(id='crime_map'),
    dcc.Graph(
        figure=px.histogram(df, x='TYPE', y='Incident_Counts', title='Crime Type Distribution'),
        id='crime_histogram'
    )
])

@app.callback(
    Output('time_series', 'figure'),
    Input('crime_type', 'value')
)
def update_time_series(selected_type):
    filtered_df = df[df['TYPE'] == selected_type]
    time_fig = px.line(filtered_df, x='Date', y='Incident_Counts', title='Crime Trend Over Time')
    return time_fig

@app.callback(
    Output('crime_bar', 'figure'),
    Input('crime_type', 'value')
)
def update_bar_chart(selected_type):
    filtered_df = df[df['TYPE'] == selected_type]
    bar_fig = px.bar(filtered_df, x='NEIGHBOURHOOD', y='Incident_Counts', title='Crime Distribution by Type')
    return bar_fig

@app.callback(
    Output('crime_map', 'figure'),
    Input('crime_type', 'value')
)
def update_map(selected_type):
    filtered_df = df[df['TYPE'] == selected_type]
    map_fig = px.scatter_mapbox(
        filtered_df, lat='Latitude', lon='Longitude', color='TYPE',
        title='Crime Heatmap by Location', mapbox_style='open-street-map'
    )
    return map_fig

if __name__ == '__main__':
    app.run(debug=False) # Change app.run_server to app.run


Crime Summary:
                                TYPE  Incident_Counts
0         Break and Enter Commercial            30044
1  Break and Enter Residential/Other            56558
2                           Mischief            63003
3                        Other Theft            43586
4                 Theft from Vehicle           152215


<IPython.core.display.Javascript object>